# 크롤링
## 쇼핑몰 목록
- [29cm](https://www.29cm.co.kr/home/)
- [MUSINSA](https://www.musinsa.com/app/)
- [ABLY](https://m.a-bly.com/)

## 카테고리
- **상의**
    - 긴팔티
        - 카라티
    - 반팔티
        - 반팔카라티
    - 민소매
        - 난닝구 포함
    - 후드티
        - 반팔 후드도 포함
    - 맨투맨
    - 니트 / 스웨터
        - 조끼도 포함
    - 셔츠 / 블라우스
        - 반팔 / 긴팔
    - 기타
        - 스포츠 딸려오면 기타로 넣기
- **하의**
    - 데님
    - 면
    - 슬랙스
    - 트레이닝 / 조거 팬츠
    - 스커트
    - 레깅스
    - 숏 팬츠
    - 기타
        - 점프 슈트 / 오버올
- **아우터**
    - 숏패딩 / 패딩조끼
    - 롱패딩
    - 숏코트
    - 롱코트
    - 라이더 재킷
    - 블레이저
    - 무스탕
    - 재킷
        - 청자켓 포함
    - 점퍼
    - 플리스
    - 가디건
    - 후드집업
    - 바람막이
    - 기타
- **원피스**
    - 롱원피스
    - 미니원피스
    - 기타
- **가방**
    - 크로스백
    - 숄더백
    - 토트백
        - 더플백 포함
    - 클러치
    - 에코 / 캔버스 백
    - 백팩
    - 웨이스트백
    - 기타
- **신발**
    - 스니커즈
    - 러닝화 / 워킹화
    - 스포츠화
        - 농구, 축구
    - 구두
    - 힐 / 펌프스
    - 로퍼
    - 뮬 / 블로퍼
    - 플랫 슈즈
    - 부츠
    - 샌들 / 슬리퍼
        - 크록스도 포함
    - 기타
- **모자**
    - 볼캡 / 야구모자
    - 스냅백
    - 비니
    - 버킷햇
    - 베레모
    - 페도라
    - 기타